# Chapter 12: Agentic Workflows Demo

This notebook demonstrates the core agentic patterns and workflows from Chapter 12, showing how to build autonomous AI agents that can think, act, observe, and respond.

## Learning Objectives
- Understand the Agentic Loop (THINK → ACT → OBSERVE → RESPOND)
- Implement context management for long conversations
- Build tool-calling capabilities
- Create robust error handling patterns
- Design multi-agent collaboration systems


## 2. Setup and Imports

First, let's import the agentic patterns module and set up our environment.


In [1]:
import sys
import os
sys.path.append('..')

from agentic_patterns import AgenticLoop, ContextPruner, SimpleAgent, AgentState, ToolCall, AgentMessage
import time
import json
from typing import List, Dict, Any


## 3. The Agentic Loop in Action

Let's demonstrate the core THINK → ACT → OBSERVE → RESPOND cycle that enables autonomous behavior.


In [2]:
# Create a simple agent
agent = SimpleAgent()

print("=== Agentic Loop Demo ===")
print("Let's see the agentic loop in action:\n")

# Demo the agentic loop
user_queries = [
    "Hello! Can you help me read a file called 'notes.txt'?",
    "What's the weather like today?",
    "Search for documents about machine learning",
    "Write a summary of our conversation"
]

for i, query in enumerate(user_queries, 1):
    print(f"\n--- Interaction {i} ---")
    print(f"User: {query}")
    
    # Show the agentic loop states
    print(f"Agent State: {agent.agentic_loop.state.value}")
    
    response = agent.chat(query)
    print(f"Agent: {response}")
    
    # Show context information
    print(f"Context: {len(agent.agentic_loop.messages)} messages, {agent.agentic_loop.current_tokens} tokens")
    
    time.sleep(1)  # Pause for readability


=== Agentic Loop Demo ===
Let's see the agentic loop in action:


--- Interaction 1 ---
User: Hello! Can you help me read a file called 'notes.txt'?
Agent State: thinking
Agent: I've executed the requested action. Result: File contents: This is a sample file with some text content.
Context: 4 messages, 10 tokens

--- Interaction 2 ---
User: What's the weather like today?
Agent State: responding
Agent: I understand your request. How can I help you further?
Context: 6 messages, 15 tokens

--- Interaction 3 ---
User: Search for documents about machine learning
Agent State: responding
Agent: I've executed the requested action. Result: Found 3 relevant documents about the topic.
Context: 10 messages, 21 tokens

--- Interaction 4 ---
User: Write a summary of our conversation
Agent State: responding
Agent: I've executed the requested action. Result: File written successfully.
Context: 14 messages, 27 tokens


## 4. Context Management Deep Dive

Let's explore how the tier-based context optimization works in practice.


In [3]:
# Create a context pruner with smaller limits for demo
pruner = ContextPruner(max_tokens=100, buffer=20)

print("=== Context Management Demo ===")
print("Simulating a long conversation to trigger context pruning:\n")

# Simulate a long conversation
messages = []
current_tokens = 0

# Add system prompt
system_msg = pruner.system_prompt
messages.append(system_msg)
current_tokens += 50  # Approximate token count

print(f"Initial: {len(messages)} messages, {current_tokens} tokens")
print(f"Usage: {current_tokens/pruner.max_tokens*100:.1f}%")

# Add conversation messages
conversation_messages = [
    "Hello, I need help with my project",
    "I can help you with that. What kind of help do you need?",
    "I want to analyze some data files",
    "I can help you analyze data. What type of files do you have?",
    "I have CSV files with sales data",
    "Great! I can help you analyze CSV files. Let me read the data first.",
    "Can you also create a summary report?",
    "Absolutely! I'll analyze the data and create a comprehensive report."
]

for i, content in enumerate(conversation_messages):
    role = "user" if i % 2 == 0 else "assistant"
    msg = AgentMessage(role=role, content=content)
    messages.append(msg)
    current_tokens += len(content.split())
    
    print(f"\nAfter message {i+1}: {len(messages)} messages, {current_tokens} tokens")
    print(f"Usage: {current_tokens/pruner.max_tokens*100:.1f}%")
    
    # Show pruning decision
    if current_tokens > pruner.max_tokens * 0.5:
        pruned = pruner.prune_context(messages, current_tokens)
        print(f"Pruned to: {len(pruned)} messages")
        print(f"Pruning strategy: {'Fresh' if current_tokens/pruner.max_tokens < 0.5 else 'Moderate' if current_tokens/pruner.max_tokens < 0.75 else 'Compressed' if current_tokens/pruner.max_tokens < 0.9 else 'Critical'}")


=== Context Management Demo ===
Simulating a long conversation to trigger context pruning:

Initial: 1 messages, 50 tokens
Usage: 50.0%

After message 1: 2 messages, 57 tokens
Usage: 57.0%
Pruned to: 3 messages
Pruning strategy: Moderate

After message 2: 3 messages, 70 tokens
Usage: 70.0%
Pruned to: 3 messages
Pruning strategy: Moderate

After message 3: 4 messages, 77 tokens
Usage: 77.0%
Pruned to: 6 messages
Pruning strategy: Compressed

After message 4: 5 messages, 90 tokens
Usage: 90.0%
Pruned to: 1 messages
Pruning strategy: Critical

After message 5: 6 messages, 97 tokens
Usage: 97.0%
Pruned to: 2 messages
Pruning strategy: Critical

After message 6: 7 messages, 111 tokens
Usage: 111.0%
Pruned to: 1 messages
Pruning strategy: Critical

After message 7: 8 messages, 118 tokens
Usage: 118.0%
Pruned to: 2 messages
Pruning strategy: Critical

After message 8: 9 messages, 128 tokens
Usage: 128.0%
Pruned to: 1 messages
Pruning strategy: Critical


## 5. Tool Calling Patterns

Let's implement and test different tool calling patterns.


In [4]:
class ToolRegistry:
    """Registry for available tools"""
    
    def __init__(self):
        self.tools = {}
        self._register_default_tools()
    
    def _register_default_tools(self):
        """Register default tools"""
        self.tools = {
            "read_file": self._read_file,
            "write_file": self._write_file,
            "search_documents": self._search_documents,
            "list_files": self._list_files,
            "calculate": self._calculate
        }
    
    def _read_file(self, path: str) -> str:
        """Read file contents"""
        try:
            with open(path, 'r') as f:
                return f.read()
        except FileNotFoundError:
            return f"Error: File '{path}' not found"
        except Exception as e:
            return f"Error reading file: {str(e)}"
    
    def _write_file(self, path: str, content: str) -> str:
        """Write content to file"""
        try:
            with open(path, 'w') as f:
                f.write(content)
            return f"Successfully wrote {len(content)} characters to '{path}'"
        except Exception as e:
            return f"Error writing file: {str(e)}"
    
    def _search_documents(self, query: str, top_k: int = 5) -> str:
        """Search documents (simulated)"""
        # Simulated search results
        results = [
            f"Document 1: Contains information about {query}",
            f"Document 2: Related to {query} concepts",
            f"Document 3: Advanced {query} techniques"
        ]
        return f"Found {len(results)} documents: {', '.join(results[:top_k])}"
    
    def _list_files(self, directory: str = ".") -> str:
        """List files in directory"""
        try:
            files = os.listdir(directory)
            return f"Files in {directory}: {', '.join(files[:10])}"
        except Exception as e:
            return f"Error listing files: {str(e)}"
    
    def _calculate(self, expression: str) -> str:
        """Calculate mathematical expression"""
        try:
            # Simple calculation (unsafe in production!)
            result = eval(expression)
            return f"Result: {result}"
        except Exception as e:
            return f"Error calculating: {str(e)}"
    
    def execute_tool(self, tool_name: str, parameters: dict) -> str:
        """Execute a tool with parameters"""
        if tool_name not in self.tools:
            return f"Error: Tool '{tool_name}' not found"
        
        try:
            return self.tools[tool_name](**parameters)
        except Exception as e:
            return f"Error executing tool: {str(e)}"

# Test tool registry
tool_registry = ToolRegistry()

print("=== Tool Calling Demo ===")
print("Available tools:", list(tool_registry.tools.keys()))

# Test different tools
test_calls = [
    ("list_files", {"directory": "."}),
    ("search_documents", {"query": "machine learning", "top_k": 3}),
    ("calculate", {"expression": "2 + 2 * 3"}),
    ("write_file", {"path": "test_output.txt", "content": "This is a test file created by the agent."})
]

for tool_name, params in test_calls:
    print(f"\nCalling {tool_name} with {params}")
    result = tool_registry.execute_tool(tool_name, params)
    print(f"Result: {result}")


=== Tool Calling Demo ===
Available tools: ['read_file', 'write_file', 'search_documents', 'list_files', 'calculate']

Calling list_files with {'directory': '.'}
Result: Files in .: hero-project, requirements.txt, agentic_workflows.ipynb, __pycache__, README.md, agentic_patterns.py, .ipynb_checkpoints, test_output.txt, venv, setup_and_test.sh

Calling search_documents with {'query': 'machine learning', 'top_k': 3}
Result: Found 3 documents: Document 1: Contains information about machine learning, Document 2: Related to machine learning concepts, Document 3: Advanced machine learning techniques

Calling calculate with {'expression': '2 + 2 * 3'}
Result: Result: 8

Calling write_file with {'path': 'test_output.txt', 'content': 'This is a test file created by the agent.'}
Result: Successfully wrote 41 characters to 'test_output.txt'


## 6. Error Handling and Recovery

Let's implement robust error handling patterns for agentic systems.


In [5]:
class RobustAgent:
    """Agent with robust error handling and recovery"""
    
    def __init__(self, tool_registry: ToolRegistry):
        self.tool_registry = tool_registry
        self.max_retries = 3
        self.error_history = []
    
    def execute_with_recovery(self, tool_name: str, parameters: dict) -> str:
        """Execute tool with automatic error recovery"""
        attempts = 0
        errors = []
        
        while attempts < self.max_retries:
            try:
                result = self.tool_registry.execute_tool(tool_name, parameters)
                
                # Check if result indicates an error
                if result.startswith("Error:"):
                    raise Exception(result)
                
                return result
                
            except Exception as e:
                attempts += 1
                errors.append(str(e))
                
                print(f"Attempt {attempts} failed: {str(e)}")
                
                # Try to recover
                if attempts < self.max_retries:
                    parameters = self._attempt_recovery(tool_name, parameters, str(e))
                    print(f"Recovery attempt: Modified parameters to {parameters}")
        
        # All attempts failed
        self.error_history.extend(errors)
        return f"Failed after {self.max_retries} attempts. Errors: {'; '.join(errors)}"
    
    def _attempt_recovery(self, tool_name: str, parameters: dict, error: str) -> dict:
        """Attempt to recover from error by modifying parameters"""
        # Simple recovery strategies
        if "not found" in error.lower():
            if "path" in parameters:
                # Try alternative path
                original_path = parameters["path"]
                if original_path.endswith(".txt"):
                    parameters["path"] = original_path.replace(".txt", ".md")
                elif original_path.endswith(".md"):
                    parameters["path"] = original_path.replace(".md", ".txt")
        
        elif "permission" in error.lower():
            # Try with different permissions or alternative approach
            if tool_name == "write_file":
                parameters["path"] = f"temp_{parameters['path']}"
        
        return parameters
    
    def get_error_summary(self) -> str:
        """Get summary of recent errors"""
        if not self.error_history:
            return "No recent errors"
        
        recent_errors = self.error_history[-5:]  # Last 5 errors
        return f"Recent errors: {'; '.join(recent_errors)}"

# Test robust agent
robust_agent = RobustAgent(tool_registry)

print("=== Error Handling Demo ===")

# Test with various scenarios
test_scenarios = [
    ("read_file", {"path": "nonexistent.txt"}),
    ("write_file", {"path": "/root/forbidden.txt", "content": "test"}),
    ("calculate", {"expression": "1/0"}),
    ("list_files", {"directory": "/nonexistent"})
]

for tool_name, params in test_scenarios:
    print(f"\nTesting {tool_name} with {params}")
    result = robust_agent.execute_with_recovery(tool_name, params)
    print(f"Final result: {result}")

print(f"\nError summary: {robust_agent.get_error_summary()}")


=== Error Handling Demo ===

Testing read_file with {'path': 'nonexistent.txt'}
Attempt 1 failed: Error: File 'nonexistent.txt' not found
Recovery attempt: Modified parameters to {'path': 'nonexistent.md'}
Attempt 2 failed: Error: File 'nonexistent.md' not found
Recovery attempt: Modified parameters to {'path': 'nonexistent.txt'}
Attempt 3 failed: Error: File 'nonexistent.txt' not found
Final result: Failed after 3 attempts. Errors: Error: File 'nonexistent.txt' not found; Error: File 'nonexistent.md' not found; Error: File 'nonexistent.txt' not found

Testing write_file with {'path': '/root/forbidden.txt', 'content': 'test'}
Final result: Error writing file: [Errno 2] No such file or directory: '/root/forbidden.txt'

Testing calculate with {'expression': '1/0'}
Final result: Error calculating: division by zero

Testing list_files with {'directory': '/nonexistent'}
Final result: Error listing files: [Errno 2] No such file or directory: '/nonexistent'

Error summary: Recent errors: Erro

## 7. Multi-Agent Collaboration

Let's implement a multi-agent system where different agents specialize in different tasks.


In [6]:
class SpecialistAgent:
    """Specialist agent for specific domains"""
    
    def __init__(self, name: str, domain: str, capabilities: list):
        self.name = name
        self.domain = domain
        self.capabilities = capabilities
        self.tool_registry = ToolRegistry()
    
    def can_handle(self, task: str) -> bool:
        """Check if this agent can handle the task"""
        task_lower = task.lower()
        return any(capability.lower() in task_lower for capability in self.capabilities)
    
    def execute_task(self, task: str) -> str:
        """Execute a task within this agent's domain"""
        return f"{self.name} ({self.domain}): Processing '{task}' using capabilities: {', '.join(self.capabilities)}"

class AgentOrchestrator:
    """Orchestrates multiple specialist agents"""
    
    def __init__(self):
        self.agents = [
            SpecialistAgent("DataAnalyst", "data", ["analysis", "statistics", "visualization"]),
            SpecialistAgent("CodeReviewer", "programming", ["code", "review", "debugging", "optimization"]),
            SpecialistAgent("ContentWriter", "writing", ["content", "documentation", "summarization"]),
            SpecialistAgent("SystemAdmin", "system", ["files", "directories", "permissions", "system"])
        ]
    
    def route_task(self, task: str) -> str:
        """Route task to appropriate specialist agent"""
        # Find the best agent for the task
        best_agent = None
        best_score = 0
        
        for agent in self.agents:
            if agent.can_handle(task):
                # Calculate match score
                score = sum(1 for cap in agent.capabilities if cap.lower() in task.lower())
                if score > best_score:
                    best_score = score
                    best_agent = agent
        
        if best_agent:
            return best_agent.execute_task(task)
        else:
            return f"No specialist agent found for task: {task}"
    
    def collaborate(self, complex_task: str) -> str:
        """Multiple agents work together on complex task"""
        # Break down complex task
        subtasks = self._decompose_task(complex_task)
        
        results = []
        for subtask in subtasks:
            result = self.route_task(subtask)
            results.append(f"Subtask '{subtask}': {result}")
        
        return f"Collaborative result:\n" + "\n".join(results)
    
    def _decompose_task(self, task: str) -> list:
        """Break down complex task into subtasks"""
        # Simple decomposition - in practice, use NLP
        if "analyze data and write report" in task.lower():
            return ["analyze data", "write report"]
        elif "review code and fix bugs" in task.lower():
            return ["review code", "fix bugs"]
        else:
            return [task]  # Single task

# Test multi-agent system
orchestrator = AgentOrchestrator()

print("=== Multi-Agent Collaboration Demo ===")
print("Available agents:")
for agent in orchestrator.agents:
    print(f"- {agent.name} ({agent.domain}): {', '.join(agent.capabilities)}")

# Test task routing
test_tasks = [
    "Analyze the sales data and create visualizations",
    "Review the Python code for bugs",
    "Write documentation for the API",
    "Check file permissions in the system",
    "Analyze data and write a comprehensive report"
]

for task in test_tasks:
    print(f"\nTask: {task}")
    
    if "and" in task.lower():
        # Complex task requiring collaboration
        result = orchestrator.collaborate(task)
    else:
        # Simple task
        result = orchestrator.route_task(task)
    
    print(f"Result: {result}")


=== Multi-Agent Collaboration Demo ===
Available agents:
- DataAnalyst (data): analysis, statistics, visualization
- CodeReviewer (programming): code, review, debugging, optimization
- ContentWriter (writing): content, documentation, summarization
- SystemAdmin (system): files, directories, permissions, system

Task: Analyze the sales data and create visualizations
Result: Collaborative result:
Subtask 'Analyze the sales data and create visualizations': DataAnalyst (data): Processing 'Analyze the sales data and create visualizations' using capabilities: analysis, statistics, visualization

Task: Review the Python code for bugs
Result: CodeReviewer (programming): Processing 'Review the Python code for bugs' using capabilities: code, review, debugging, optimization

Task: Write documentation for the API
Result: ContentWriter (writing): Processing 'Write documentation for the API' using capabilities: content, documentation, summarization

Task: Check file permissions in the system
Result:

## 8. Complete Agentic System Demo

Let's put it all together in a complete agentic system demonstration.


In [7]:
class CompleteAgenticSystem:
    """Complete agentic system integrating all components"""
    
    def __init__(self):
        self.context_pruner = ContextPruner()
        self.tool_registry = ToolRegistry()
        self.agent_orchestrator = AgentOrchestrator()
        self.messages = []
    
    def process_query(self, query: str) -> str:
        """Process user query through complete agentic system"""
        
        # Add user message
        self.messages.append({"role": "user", "content": query})
        
        # Determine if tool is needed
        if self._needs_tool(query):
            # Tool calling phase
            tool_call = self._decide_tool_usage(query)
            tool_result = self.tool_registry.execute_tool(
                tool_call["tool"], 
                tool_call["parameters"]
            )
            
            # Add tool messages
            self.messages.append({"role": "assistant", "content": "", "tool_call": tool_call})
            self.messages.append({"role": "tool", "content": tool_result})
            
            # Generate response incorporating tool result
            response = f"I've executed the requested action. Result: {tool_result}"
        else:
            # Direct response without tools
            response = f"I understand your request: '{query}'. How can I help you further?"
        
        # Add response
        self.messages.append({"role": "assistant", "content": response})
        
        return response
    
    def _needs_tool(self, query: str) -> bool:
        """Determine if query needs tool usage"""
        tool_indicators = ["read", "write", "search", "calculate", "list", "analyze"]
        return any(indicator in query.lower() for indicator in tool_indicators)
    
    def _decide_tool_usage(self, query: str) -> dict:
        """Decide which tool to use and with what parameters"""
        query_lower = query.lower()
        
        if "read" in query_lower or "file" in query_lower:
            return {"tool": "read_file", "parameters": {"path": "example.txt"}}
        elif "write" in query_lower or "create" in query_lower:
            return {"tool": "write_file", "parameters": {"path": "output.txt", "content": "Generated content"}}
        elif "search" in query_lower:
            return {"tool": "search_documents", "parameters": {"query": "example", "top_k": 5}}
        elif "list" in query_lower:
            return {"tool": "list_files", "parameters": {"directory": "."}}
        elif "calculate" in query_lower or any(op in query_lower for op in ["+", "-", "*", "/"]):
            return {"tool": "calculate", "parameters": {"expression": "2 + 2"}}
        else:
            return {"tool": "search_documents", "parameters": {"query": query, "top_k": 3}}
    
    def get_system_status(self) -> dict:
        """Get current system status"""
        return {
            "messages": len(self.messages),
            "context_usage": len(self.messages) / self.context_pruner.max_tokens * 100,
            "available_tools": list(self.tool_registry.tools.keys()),
            "specialist_agents": len(self.agent_orchestrator.agents)
        }

# Test complete system
system = CompleteAgenticSystem()

print("=== Complete Agentic System Demo ===")

# Test various queries
queries = [
    "Hello! Can you help me?",
    "I need to read a file called 'data.txt'",
    "Can you search for documents about machine learning?",
    "Write a summary of our conversation",
    "What files are in the current directory?",
    "Calculate 15 * 23"
]

for i, query in enumerate(queries, 1):
    print(f"\n--- Query {i} ---")
    print(f"User: {query}")
    
    response = system.process_query(query)
    print(f"Agent: {response}")
    
    # Show system status
    status = system.get_system_status()
    print(f"Status: {status['messages']} messages, {status['context_usage']:.1f}% context usage")

print("\n=== System Summary ===")
final_status = system.get_system_status()
print(f"Total interactions: {final_status['messages']}")
print(f"Available tools: {', '.join(final_status['available_tools'])}")
print(f"Specialist agents: {final_status['specialist_agents']}")


=== Complete Agentic System Demo ===

--- Query 1 ---
User: Hello! Can you help me?
Agent: I understand your request: 'Hello! Can you help me?'. How can I help you further?
Status: 2 messages, 0.1% context usage

--- Query 2 ---
User: I need to read a file called 'data.txt'
Agent: I've executed the requested action. Result: Error: File 'example.txt' not found
Status: 6 messages, 0.3% context usage

--- Query 3 ---
User: Can you search for documents about machine learning?
Agent: I've executed the requested action. Result: Found 3 documents: Document 1: Contains information about example, Document 2: Related to example concepts, Document 3: Advanced example techniques
Status: 10 messages, 0.5% context usage

--- Query 4 ---
User: Write a summary of our conversation
Agent: I've executed the requested action. Result: Successfully wrote 17 characters to 'output.txt'
Status: 14 messages, 0.7% context usage

--- Query 5 ---
User: What files are in the current directory?
Agent: I understand y